Integrated Local Threats to Coral Reefs	bio.024.5 http://www.wri.org/publication/reefs-risk-revisited

In [1]:
import numpy as np
import pandas as pd
import rasterio

import boto3
import requests as req

from matplotlib import pyplot as plt
%matplotlib inline
import os
import sys
import threading

In [9]:
s3_bucket = "wri-public-data"
s3_key_orig = "resourcewatch/raster/bio_024_5_coral_reef_integrated_local_threats/bio_024_5_coral_reef_integrated_local_threats.tif"

temp_folder = "/Users/nathansuberi/Desktop/RW_Data/Rasters/"
local_orig = temp_folder + "bio_024_5.tif"

s3_upload = boto3.client('s3')
s3_download = boto3.resource('s3')

#s3.meta.client.download_file(s3_bucket, s3_key_orig, local_orig)

In [6]:
s3_key_edit = s3_key_orig[:-4] + "_edit.tif"

In [8]:
local_edit = temp_folder + "bio_024_5_edit.tif"

with rasterio.open(local_orig) as src:
    kwargs = src.profile
    kwargs.update(
        driver='GTiff',
        dtype=rasterio.int16,  #rasterio.int16, rasterio.int32, rasterio.uint8,rasterio.uint16, rasterio.uint32, rasterio.float32, rasterio.float64
        count=1,
        compress='lzw',
        nodata=0,
        bigtiff='NO',
        crs = 'EPSG:4326', 
    )
    data = src.read()
    
    with rasterio.open(local_edit, 'w', **kwargs) as dst:
        dst.write(data.astype(kwargs["dtype"]))

/Users/nathansuberi/.pyenv/versions/3.6.0/lib/python3.6/site-packages/rasterio/__init__.py:160: FutureWarning: GDAL-style transforms are deprecated and will not be supported in Rasterio 1.0.
  transform = guard_transform(transform)


In [10]:
# Helper function to view upload progress
class ProgressPercentage(object):
        def __init__(self, filename):
            self._filename = filename
            self._size = float(os.path.getsize(filename))
            self._seen_so_far = 0
            self._lock = threading.Lock()

        def __call__(self, bytes_amount):
            # To simplify we'll assume this is hooked up
            # to a single filename.
            with self._lock:
                self._seen_so_far += bytes_amount
                percentage = (self._seen_so_far / self._size) * 100
                sys.stdout.write(
                    "\r%s  %s / %s  (%.2f%%)" % (
                        self._filename, self._seen_so_far, self._size,
                        percentage))
                sys.stdout.flush()

In [11]:
s3_upload.upload_file(local_edit, s3_bucket, s3_key_edit, Callback=ProgressPercentage(local_edit))

/Users/nathansuberi/Desktop/RW_Data/Rasters/bio_024_5_edit.tif  54701542 / 54701542.0  (100.00%)